<a href="https://colab.research.google.com/github/ricardogr07/trabajo-final-aprendizaje/blob/definir_dataset/src/Analisis_ML_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de dataset

## Importar librerías

In [ ]:
%pip install pandas numpy ucimlrepo

In [1]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np
# ....

In [6]:
!git clone https://github.com/ricardogr07/trabajo-final-aprendizaje.git
%cd trabajo-final-aprendizaje/src
from evaluation_functions import create_pipelines, evaluate_pipelines, display_metrics

Cloning into 'trabajo-final-aprendizaje'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 3), reused 13 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 155.50 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/trabajo-final-aprendizaje/src/trabajo-final-aprendizaje/src/trabajo-final-aprendizaje/src


## Carga y Preprocesamiento de Datos

En esta sección se carga el conjunto de datos del archivo CSV que contiene información...

### Carga del dataset

In [6]:
# Cargar el dataset 1
# https://archive.ics.uci.edu/dataset/45/heart+disease
file_name = '../heart_disease_clevelad_data.csv'

df = pd.read_csv(file_name)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  target    303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


In [11]:
# Cargar el dataset 2
# https://archive.ics.uci.edu/dataset/27/credit+approval
from ucimlrepo import fetch_ucirepo
# fetch dataset
credit_approval = fetch_ucirepo(id=27)

# data (as pandas dataframes)
X = credit_approval.data.features
y = credit_approval.data.targets

# metadata
print(credit_approval.metadata.name)
print(credit_approval.metadata.repository_url)
print(credit_approval.metadata.abstract)
# variable information
print(credit_approval.variables)

Credit Approval
https://archive.ics.uci.edu/dataset/27/credit+approval
This data concerns credit card applications; good mix of attributes
   name     role         type demographic description units missing_values
0   A16   Target  Categorical        None        None  None             no
1   A15  Feature   Continuous        None        None  None             no
2   A14  Feature   Continuous        None        None  None            yes
3   A13  Feature  Categorical        None        None  None             no
4   A12  Feature  Categorical        None        None  None             no
5   A11  Feature   Continuous        None        None  None             no
6   A10  Feature  Categorical        None        None  None             no
7    A9  Feature  Categorical        None        None  None             no
8    A8  Feature   Continuous        None        None  None             no
9    A7  Feature  Categorical        None        None  None            yes
10   A6  Feature  Categorical       

### Exploración inicial del dataset

### Preprocesar Datos

## Análisis exploratorio de datos


## Selección de modelo usando técnicas de aprendizaje supervisado


### Selección de características


### Validación de balanceo de clases


### Selección de modelo supervisado


### Ajuste de hiperparámetros del modelo


### Evaluación del modelo seleccionado


## Análisis de clúster con aprendizaje no supervisado


### PCA


### Clustering
